In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import nltk
from nltk.util import ngrams
%matplotlib inline

In [15]:
train = pd.read_csv('train.csv')

Analysing the top Tri-and Bi grams associated to each author. (Provavelmente será substituído pelo peso do tf-idf)

In [16]:
def generate_ngrams(text, n=2):
    words = text.split()
    iterations = len(words) - 1
    for i in range(iterations):
        yield words[i:i + n]

## Generating network of Trigram counts

Obtendo as tabelas para cada autor...

In [17]:
# DataFrame for Mary Shelley
ngrams = {}
for title in train[train.author=="MWS"]['text']:
        for ngram in generate_ngrams(title, 3):
            ngram = ' '.join(ngram)
            if ngram in ngrams:
                ngrams[ngram] += 1
            else:
                ngrams[ngram] = 1

ngrams_mws_df = pd.DataFrame.from_dict(ngrams, orient='index')
ngrams_mws_df.columns = ['count']
ngrams_mws_df['author'] = 'Mary Shelley'
ngrams_mws_df.reset_index(level=0, inplace=True)

# DataFrame for Edgar Allen Poe
ngrams = {}
for title in train[train.author=="EAP"]['text']:
        for ngram in generate_ngrams(title, 3):
            ngram = ' '.join(ngram)
            if ngram in ngrams:
                ngrams[ngram] += 1
            else:
                ngrams[ngram] = 1
ngrams_eap_df = pd.DataFrame.from_dict(ngrams, orient='index')
ngrams_eap_df.columns = ['count']
ngrams_eap_df['author'] = 'Edgar Allen Poe'
ngrams_eap_df.reset_index(level=0, inplace=True)

# DataFrame for HP Lovecraft
ngrams = {}
for title in train[train.author=="HPL"]['text']:
        for ngram in generate_ngrams(title, 3):
            ngram = ' '.join(ngram)
            if ngram in ngrams:
                ngrams[ngram] += 1
            else:
                ngrams[ngram] = 1

ngrams_hpl_df = pd.DataFrame.from_dict(ngrams, orient='index')
ngrams_hpl_df.columns = ['count']
ngrams_hpl_df['author'] = 'HP lovecraft'
ngrams_hpl_df.reset_index(level=0, inplace=True)
ngrams_eap_df.sort_values(by='count', ascending=False).head(3)

,index,count,author
3202,one of the,77,Edgar Allen Poe
3104,I could not,64,Edgar Allen Poe
6613,as well as,47,Edgar Allen Poe


In [18]:
ngrams_hpl_df.sort_values(by='count', ascending=False).head(3)

,index,count,author
1573,. .,90,HP lovecraft
1572,. . .,77,HP lovecraft
9444,out of the,48,HP lovecraft


In [19]:
ngrams_mws_df.sort_values(by='count', ascending=False).head(3)

,index,count,author
6635,I did not,39,Mary Shelley
4377,to me.,38,Mary Shelley
9930,I could not,36,Mary Shelley


## NetworkX Graphs
Returning the top 20 Trigrams attributed to each author.

In [20]:
trigram_df = pd.concat([
    ngrams_eap_df.sort_values(by='count', ascending=False).head(20),
    ngrams_hpl_df.sort_values(by='count', ascending=False).head(20),
    ngrams_mws_df.sort_values(by='count', ascending=False).head(20)
    ])

In [21]:
trigram_df

,index,count,author
3202,one of the,77,Edgar Allen Poe
3104,I could not,64,Edgar Allen Poe
6613,as well as,47,Edgar Allen Poe
1867,portion of the,43,Edgar Allen Poe
2317,would have been,42,Edgar Allen Poe
1811,might have been,40,Edgar Allen Poe
3600,that is to,37,Edgar Allen Poe
4771,that of the,37,Edgar Allen Poe
5506,which I had,35,Edgar Allen Poe
283,I had been,32,Edgar Allen Poe


NetworkX contains a function which takes in a Pandas datagrame as input where you are able to specify the source and target columns.

In [22]:
g = nx.from_pandas_edgelist(trigram_df, source='author', target='index')

In [23]:
print(nx.info(g))

Graph with 49 nodes and 60 edges


C:\Users\Felype\AppData\Local\Temp/ipykernel_13736/1432719896.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


In [24]:
len(trigram_df['count'])

60

In [25]:
plt.figure(figsize=(10, 10))
cmap = plt.cm.coolwarm
colors = [n for n in range(len(g.nodes()))]
pos = nx.spring_layout(g, k=0.14)
nx.draw_networkx(g, pos, , cmap=cmap,
node_color=colors, edge_color='grey', font_size=15, width=2, alpha=1)
plt.title('Network diagram of Top 20 Trigrams w/o Stopwords removed attributed to each Author',
fontsize=18)
plt.show()

SyntaxError: invalid syntax (Temp/ipykernel_13736/274565558.py, line 5)